# Performance Report

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from deepchecks.vision.base import VisionDataset
from torchvision import models
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
import copy

In [3]:
from deepchecks.vision.checks.performance import PerformanceReport

/home/yura/Projects/deepchecks/deepchecks/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



## Loading data and model:

In [4]:
from deepchecks.vision.datasets.classification.mnist import (
    load_model, 
    load_dataset, 
    MNistNet, 
)

In [5]:
def simple_nn():
    torch.manual_seed(42)

    # Define model
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super(NeuralNetwork, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28 * 28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10)
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

    model = NeuralNetwork().to('cpu')
    return model

def trained_mnist(simple_nn, mnist_data_loader_train):
    torch.manual_seed(42)
    simple_nn = copy.deepcopy(simple_nn)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(simple_nn.parameters(), lr=1e-3)
    size = len(mnist_data_loader_train.dataset)
    # Training 1 epoch
    simple_nn.train()
    for batch, (X, y) in enumerate(mnist_data_loader_train):
        X, y = X.to('cpu'), y.to('cpu')

        # Compute prediction error
        pred = simple_nn(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return simple_nn

In [6]:
train_ds = load_dataset(train=True, object_type='Dataset')
test_ds = load_dataset(train=False, object_type='Dataset')

In [7]:
model = trained_mnist(simple_nn(), train_ds.get_data_loader())

loss: 2.287713  [    0/60000]
loss: 2.271768  [ 6400/60000]
loss: 2.227654  [12800/60000]
loss: 2.213990  [19200/60000]
loss: 2.159032  [25600/60000]
loss: 2.118144  [32000/60000]
loss: 2.020084  [38400/60000]
loss: 1.975659  [44800/60000]
loss: 1.823154  [51200/60000]
loss: 1.817344  [57600/60000]


## Running peformance report on classification

In [8]:
check = PerformanceReport(prediction_extract=nn.Softmax(dim=1))
check.run(train_ds, test_ds, model)

Performance Report Summarize given metrics on a dataset and model. Read More... Additional Outputs

# Performance report object detection

In [9]:
from deepchecks.vision.datasets.detection import coco
import numpy as np

In [10]:
yolo = coco.get_trained_yolov5_object_detection()

Using cache found in /home/yura/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-1-31 torch 1.10.2+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [11]:
coco_dataloader = coco.get_coco_dataloader()

In [12]:
from deepchecks.vision.dataset import VisionDataset

In [13]:
train_ds = VisionDataset(coco_dataloader, label_type='object_detection', num_classes=80)

In [16]:
check = PerformanceReport(prediction_extract=coco.yolo_wrapper)
check.run(train_ds, train_ds, yolo)

/home/yura/Projects/deepchecks/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/home/yura/Projects/deepchecks/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

Engine run starting with max_epochs=1.
/home/yura/Projects/deepchecks/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/home/yura/Projects/deepchecks/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

Epoch[1] Complete. Time taken: 00:00:60
Engine run complete. Time taken: 00:00:60
/home/yura/Projects/deepchecks/deepchecks/venv/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarn

Performance Report Summarize given metrics on a dataset and model. Read More... Additional Outputs